In [222]:
import pandas as pd
import re
import string

In [223]:
pd.set_option('display.max_colwidth', -1)

In [224]:
train = pd.read_csv("train.csv",encoding = "ISO-8859-1")
train.head(15)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires,1
6,10,NaN,NaN,"#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas",1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in the woods...,1
8,14,NaN,NaN,There's an emergency evacuation happening now in the building across the street,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our area...,1


## Cuento los links de cada tweet: (tengo que hacerlo antes de limpiarlos porque en la limpieza se pierden)

In [225]:
train['cantidad_links'] = train['text'].str.count('http')

In [226]:
train.nlargest(30,'cantidad_links').head()

,id,keyword,location,text,target,cantidad_links
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
7082,10143,upheaval,Attock,Ancient Mayan Tablet found via http://t.co/LmUMzkLtln http://t.co/yebxxAryBF http://t.co/SRRUqfffr6 http://t.co/CadzxAgMSI,0,4
1060,1531,bomb,NaN,beforeitsnews : Global Derivatives: $1.5 Quadrillion Time Bomb http://t.co/GhmmUj7GbE (vÂÃ_ http://t.co/u9LvvLzhYe) http://t.co/LyJ57pq3yX,0,3
1373,1980,bush%20fires,Trinidad and Tobago,Drought fuels bush fires in Jamaica - http://t.co/ZDtDqQbAHC http://t.co/PsQCNsVfgP - @JamaicaObserver @cnewslive RE https://t.co/6ZGef8J8Bm,1,3


## Limpieza de los textos

In [227]:
train_limpio = train.copy()

In [228]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()  #quito_mayusculas
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    return text

In [229]:
train_limpio['text'] = train_limpio['text'].apply(lambda x: clean_text(x))
train_limpio.head()

,id,keyword,location,text,target,cantidad_links
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0


## Analizar las repeticiones de un tweet

In [230]:
train_limpio['repeticiones'] = train_limpio.groupby('text')['id'].transform('count')
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,repeticiones
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,1
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,1
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,1


In [231]:
train_limpio.nlargest(20,'repeticiones').head()

,id,keyword,location,text,target,cantidad_links,repeticiones
5876,8393,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,22
5877,8394,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,22
5879,8397,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,22
5882,8401,sandstorm,hkXfYMhEx,watch this airport get swallowed up by a sandstorm in under a minute,1,1,22
5883,8402,sandstorm,US,watch this airport get swallowed up by a sandstorm in under a minute,1,1,22


## Remuevo los datos repetidos

In [232]:
train_limpio = train_limpio[['text','target','cantidad_links','repeticiones']]
train_limpio.shape

(7613, 4)

In [233]:
train_limpio.head()

,text,target,cantidad_links,repeticiones
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,1
1,forest fire near la ronge sask canada,1,0,1
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,1
3,people receive wildfires evacuation orders in california,1,0,1
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,1


In [234]:
train_limpio = train_limpio.drop_duplicates()
train_limpio.shape

(7131, 4)

# Longitud de los tweets

In [235]:
import math

def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [236]:
train_limpio['longitud_tweet'] = train_limpio['text'].apply(lambda x: roundup(len(x)))

In [238]:
train_limpio

,text,target,cantidad_links,repeticiones,longitud_tweet
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,1,70
1,forest fire near la ronge sask canada,1,0,1,40
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,1,130
3,people receive wildfires evacuation orders in california,1,0,1,60
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,1,90
5,rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires,1,0,1,110
6,flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas,1,0,1,100
7,im on top of the hill and i can see a fire in the woods,1,0,1,60
8,theres an emergency evacuation happening now in the building across the street,1,0,1,80
9,im afraid that the tornado is coming to our area,1,0,1,50


## Caso abarcativo: agrupo por palabras y me quedon con las mas repetidas

In [239]:
import re 
from string import punctuation

palabras_por_separado = train_limpio.copy()
palabras_por_separado.set_index(['target'],inplace=True)
palabras_por_separado = palabras_por_separado['text'].str.split(expand=True).stack().to_frame()

In [240]:
palabras_por_separado.columns = ['word']
palabras_por_separado['word'] = palabras_por_separado['word'].str.replace(rf'[{punctuation}]', '')
palabras_por_separado.head()

word
target          
1      0  our   
       1  deeds 
       2  are   
       3  the   
       4  reason

In [241]:
palabras_por_separado.reset_index(inplace=True)
del palabras_por_separado['level_1']
palabras_por_separado.head()

,target,word
0,1,our
1,1,deeds
2,1,are
3,1,the
4,1,reason


In [243]:
palabras_mas_comunes = palabras_por_separado.groupby('word')['target'].count().rename('cantidad_apariciones').reset_index().nlargest(400,'cantidad_apariciones')
palabras_mas_comunes.head()

,word,cantidad_apariciones
15299,the,3070
0,a,2035
15570,to,1853
7907,in,1816
10982,of,1674


In [244]:
palabras_mas_comunes.word.values

array(['the', 'a', 'to', 'in', 'of', 'and', 'i', 'is', 'on', 'for', 'you',
       'my', 'it', 'that', 'with', 'at', 'this', 'by', 'be', 'are', 'was',
       'from', 'have', 'like', 'me', 'but', 'just', 'so', 'im', 'not',
       'amp', 'as', 'up', 'your', 'out', 'its', 'all', 'will', 'no',
       'fire', 'an', 'when', 'has', 'if', 'we', 'after', 'now', 'get',
       'about', 'new', 'or', 'they', 'what', 'dont', 'via', 'how', 'one',
       'he', 'people', 'more', 'news', 'do', 'been', 'us', 'can',
       'emergency', 'over', 'video', 'there', 'who', 'into', 'were',
       'police', 'her', 'would', 'his', 'disaster', 'some', 'body',
       'back', 'storm', 'still', 'burning', 'than', 'got', 'had', 'know',
       'off', 'them', 'why', 'man', 'day', 'crash', 'time', 'see',
       'buildings', 'first', 'rt', 'cant', 'going', 'california', 'our',
       'world', 'love', 'nuclear', 'suicide', 'fires', 'go', 'youtube',
       'attack', 'dead', 'their', 'bomb', 'only', 'good', 'being', 'car',
  

In [245]:
for i in palabras_mas_comunes.word.values:
    train_limpio[i]= train['text'].apply(lambda x: x.count(i) )

# Remocion de StopWords

In [246]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
len(stop)

[nltk_data] Downloading package stopwords to /home/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [247]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [248]:
train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x.split() if item not in stop])
train_limpio.head()

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,traumatised,wanna,air,airplane,bag,blew,blown,bridge,crush,derailment
0,"[deeds, reason, earthquake, may, allah, forgive, us]",1,0,1,70,1,6,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,1,40,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[residents, asked, shelter, place, notified, officers, evacuation, shelter, place, orders, expected]",1,0,1,130,1,7,1,3,1,...,0,0,0,0,0,0,0,0,0,0
3,"[people, receive, wildfires, evacuation, orders, california]",1,0,1,60,0,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"[got, sent, photo, ruby, alaska, smoke, wildfires, pours, school]",1,0,1,90,0,4,2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [249]:
# Como volver a convertirlo en texto (no lo voy a hacer por ahora)
# train_limpio['text'] = train_limpio['text'].apply(lambda x: ' '.join(x))
# train_limpio.head()

# Correccion de typos (me conviene correjirlos o simplemente sacarlos y agregar en otra columna "cant palabras mal escritas"?)

## Opcion 1: las corrijo

In [250]:
train_limpio.head()

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,traumatised,wanna,air,airplane,bag,blew,blown,bridge,crush,derailment
0,"[deeds, reason, earthquake, may, allah, forgive, us]",1,0,1,70,1,6,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,1,40,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[residents, asked, shelter, place, notified, officers, evacuation, shelter, place, orders, expected]",1,0,1,130,1,7,1,3,1,...,0,0,0,0,0,0,0,0,0,0
3,"[people, receive, wildfires, evacuation, orders, california]",1,0,1,60,0,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"[got, sent, photo, ruby, alaska, smoke, wildfires, pours, school]",1,0,1,90,0,4,2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [251]:
pruebita = train_limpio.iloc[1:150].copy()

In [252]:
pruebita

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,traumatised,wanna,air,airplane,bag,blew,blown,bridge,crush,derailment
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,1,40,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[residents, asked, shelter, place, notified, officers, evacuation, shelter, place, orders, expected]",1,0,1,130,1,7,1,3,1,...,0,0,0,0,0,0,0,0,0,0
3,"[people, receive, wildfires, evacuation, orders, california]",1,0,1,60,0,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"[got, sent, photo, ruby, alaska, smoke, wildfires, pours, school]",1,0,1,90,0,4,2,1,0,...,0,0,0,0,0,0,0,0,0,0
5,"[rockyfire, update, california, hwy, closed, directions, due, lake, county, fire, cafire, wildfires]",1,0,1,110,0,4,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,"[flood, disaster, heavy, rain, causes, flash, flooding, streets, manitou, colorado, springs, areas]",1,0,1,100,0,9,1,4,1,...,0,0,0,0,0,0,0,0,0,0
7,"[im, top, hill, see, fire, woods]",1,0,1,60,2,3,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,"[theres, emergency, evacuation, happening, building, across, street]",1,0,1,80,2,5,0,3,0,...,0,0,0,0,0,0,0,0,0,0
9,"[im, afraid, tornado, coming, area]",1,0,1,50,1,6,2,1,0,...,0,0,0,0,0,0,0,0,0,0
10,"[three, people, died, heat, wave, far]",1,0,1,50,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [253]:
#pruebita['text'] = pruebita['text'].apply(lambda x: TextBlob(x).correct() )
pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))

NameError: name 'TextBlob' is not defined

###  Muuy lento, como 3 minutos en corregir typos de apenas 150 filas

# Opcion 2: contar cuantas palabras mal escritas tiene cada texto

In [254]:
from nltk.corpus import brown
nltk.download('brown')
word_list = brown.words()
word_set = set(word_list)

[nltk_data] Downloading package brown to /home/andres/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [255]:
len(word_list)

1161192

In [256]:
def cant_misspelled(x):
    misspelled_words = 0
    for i in x:
        if (i not in word_list):
            misspelled_words=misspelled_words+1
    return(misspelled_words)

In [257]:
x=['hello', 'there', 'general', 'nto']
x

['hello', 'there', 'general', 'nto']

In [258]:
cant_misspelled(x)

1

### Mas lento todavia...  pruebo otra cosa

## Opcion 3 (no me resuelve todos los typos pero capaz si los intencionales)
## Contar la cantidad de veces seguidas que se repite una letra, si alguna se repite mas de 3 veces, es probable que haya sido intencional, lo cual le quita credibildad

In [259]:
train_limpio.head()

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,traumatised,wanna,air,airplane,bag,blew,blown,bridge,crush,derailment
0,"[deeds, reason, earthquake, may, allah, forgive, us]",1,0,1,70,1,6,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,1,40,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[residents, asked, shelter, place, notified, officers, evacuation, shelter, place, orders, expected]",1,0,1,130,1,7,1,3,1,...,0,0,0,0,0,0,0,0,0,0
3,"[people, receive, wildfires, evacuation, orders, california]",1,0,1,60,0,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"[got, sent, photo, ruby, alaska, smoke, wildfires, pours, school]",1,0,1,90,0,4,2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [260]:
def letras_repetidas(text):
    max_repeteated_letters = 0
    current_repeated_letters = 1
    for word in text:
        if(current_repeated_letters>max_repeteated_letters):
            max_repeteated_letters=current_repeated_letters
        current_repeated_letters=1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeteated_letters):
                    max_repeteated_letters=current_repeated_letters
                current_repeated_letters=1
    return (max_repeteated_letters)

In [261]:
def remover_palabras_con_letras_repetidas(text):
    max_repeated_letters = 0
    current_repeated_letters = 1
    new_text = []
    for word in text:
        current_repeated_letters=1
        max_repeated_letters = 1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeated_letters):
                    max_repeated_letters=current_repeated_letters
                current_repeated_letters=1
        if(current_repeated_letters<2 and max_repeated_letters<2):
            new_text.append(word)
    return (new_text)

In [262]:
train_limpio['letras_seguidas'] = train_limpio['text'].apply(lambda x: letras_repetidas(x))

In [263]:
#Remuevo esas palabras
train_limpio['text'] = train_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x))

In [264]:
train_limpio.head()

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,wanna,air,airplane,bag,blew,blown,bridge,crush,derailment,letras_seguidas
0,"[reason, earthquake, may, forgive, us]",1,0,1,70,1,6,0,0,1,...,0,0,0,0,0,0,0,0,0,2
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,1,40,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"[residents, asked, shelter, place, notified, evacuation, shelter, place, orders, expected]",1,0,1,130,1,7,1,3,1,...,0,0,0,0,0,0,0,0,0,2
3,"[people, receive, wildfires, evacuation, orders, california]",1,0,1,60,0,4,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,"[got, sent, photo, ruby, alaska, smoke, wildfires, pours]",1,0,1,90,0,4,2,1,0,...,0,0,0,0,0,0,0,0,0,2


In [265]:
len(train_limpio[train_limpio['letras_seguidas']>2])

124

In [266]:
train_limpio[train_limpio['letras_seguidas']>2].target.mean()

0.3467741935483871

In [267]:
train_limpio.head()

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,wanna,air,airplane,bag,blew,blown,bridge,crush,derailment,letras_seguidas
0,"[reason, earthquake, may, forgive, us]",1,0,1,70,1,6,0,0,1,...,0,0,0,0,0,0,0,0,0,2
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,1,40,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"[residents, asked, shelter, place, notified, evacuation, shelter, place, orders, expected]",1,0,1,130,1,7,1,3,1,...,0,0,0,0,0,0,0,0,0,2
3,"[people, receive, wildfires, evacuation, orders, california]",1,0,1,60,0,4,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,"[got, sent, photo, ruby, alaska, smoke, wildfires, pours]",1,0,1,90,0,4,2,1,0,...,0,0,0,0,0,0,0,0,0,2


## Parece dar buenos resultados pero obviamente no reemplaza un programa que me detecte cualquier error ortografico

## Columna si contiene insultos, o la cantidad que tiene

In [268]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
insults_list = ['idiot','fuck','moron','scum','bitch','shit','bastard','cunt','ridicul','tramp']
mean_insult = [train[train['text'].str.contains(i)].target.mean() for i in insults_list]
times_insult_used = [len(train[train['text'].str.contains(i)]) for i in insults_list]

In [269]:
print(insults_list)
print(times_insult_used)
print(mean_insult)

['idiot', 'fuck', 'moron', 'scum', 'bitch', 'shit', 'bastard', 'cunt', 'ridicul', 'tramp']
[4, 67, 1, 1, 10, 57, 1, 4, 4, 1]
[0.5, 0.208955223880597, 0.0, 0.0, 0.0, 0.3157894736842105, 0.0, 0.0, 0.25, 1.0]


In [270]:
def insults(text):
    insults = ['idiot','fuck','moron','scum','bitch','shit','bastard','cunt','ridicul','tramp']
    amount = 0
    for word in text:
        if(word in insults):
            amount = amount +1
    return(amount)

In [271]:
train_limpio['insultos_por_tweet'] = train_limpio['text'].apply(lambda x: insults(x) )

In [272]:
train_limpio[train_limpio['insultos_por_tweet']==1].target.mean()

0.25773195876288657

In [273]:
# Si bien algunos aparecen mucho menos que otros, en todos se cumple que estan por debajo de la media

In [274]:
# Remuevo estas palabras de los textos
train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x if item not in insults_list])

In [275]:
train_limpio.head()

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,air,airplane,bag,blew,blown,bridge,crush,derailment,letras_seguidas,insultos_por_tweet
0,"[reason, earthquake, may, forgive, us]",1,0,1,70,1,6,0,0,1,...,0,0,0,0,0,0,0,0,2,0
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,1,40,0,6,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,"[residents, asked, shelter, place, notified, evacuation, shelter, place, orders, expected]",1,0,1,130,1,7,1,3,1,...,0,0,0,0,0,0,0,0,2,0
3,"[people, receive, wildfires, evacuation, orders, california]",1,0,1,60,0,4,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,"[got, sent, photo, ruby, alaska, smoke, wildfires, pours]",1,0,1,90,0,4,2,1,0,...,0,0,0,0,0,0,0,0,2,0


## Remover expresiones

In [276]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
expressions = ['lol','omg','wtf','lmao','imo']  #agregar mas,no importa que el train no tenga, si el test si,joya
mean_expressions = [train[train['text'].str.contains(i)].target.mean() for i in expressions]
times_expressions_used = [len(train[train['text'].str.contains(i)]) for i in expressions]

In [277]:
print(expressions)
print(times_expressions_used)
print(mean_expressions)

['lol', 'omg', 'wtf', 'lmao', 'imo']
[54, 6, 8, 14, 15]
[0.12962962962962962, 0.3333333333333333, 0.375, 0.0, 0.3333333333333333]


In [278]:
def expressions_counter(text):
    expressions = ['lol','omg','wtf','lmao','imo']
    amount = 0
    for word in text:
        if(word in expressions):
            amount = amount +1
    return(amount)

In [279]:
train_limpio['expresiones_por_tweet'] = train_limpio['text'].apply(lambda x: expressions_counter(x) )

In [280]:
# Remuevo estas palabras de los textos
train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x if item not in expressions])

## Cuento pronombres personales,esto se habia hecho en el TP1 y dio buenos resultados

In [281]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
personal_pronouns = ['i','me','you','she','her','him','he','his','we','us','them','they','im']  #agregar mas,no importa que el train no tenga, si el test si,joya
mean_personal_pronouns = [train[train['text'].str.contains(i)].target.mean() for i in personal_pronouns]
times_personal_pronoun_used = [len(train[train['text'].str.contains(i)]) for i in personal_pronouns]

In [282]:
print(personal_pronouns)
print(mean_personal_pronouns)
print(times_personal_pronoun_used)

['i', 'me', 'you', 'she', 'her', 'him', 'he', 'his', 'we', 'us', 'them', 'they', 'im']
[0.4389975213439824, 0.43219954648526077, 0.2056921086675291, 0.4017857142857143, 0.42958579881656805, 0.7189189189189189, 0.4109311740890688, 0.37099494097807756, 0.4495515695067265, 0.4447797223898612, 0.3557692307692308, 0.3006134969325153, 0.5167630057803468]
[7262, 2205, 773, 224, 845, 185, 3952, 593, 892, 1657, 104, 163, 865]


In [283]:
for i in personal_pronouns:
    train_limpio[i]= train['text'].apply(lambda x: x.count(i) )

In [284]:
#los terminos aparecen tantas veces que no me conviene, pasa estos 
# casos creo que es mejor que el CountVectorizer se encargue
# El hecho de que cada termino aparezca tantas veces, indica que cada uno puede obtener su valor por si mismo
# sin necesidad de agruparlo con otros para que lleguen a una cantidad importante.
# X ej: el algoritmo nunca va a saber reconocer el peso de un link porque todos son 
# distintos, asique los tengo que agrupar

## Palabras mas comunes (esta parte no se si deberia quedar así basandome en las del TP1 o rehacer eso ahora que está mas limpio el train o hacerlo de otra forma)

In [285]:
palabras = ['hiroshima','bombing','kill','california','suicide','fire','forest','train','accident','disaster'\
                ,'new','crash','storm','police','nuclear','two','fire','dead','died','yeas']
for i in palabras:
    train_limpio[i]= train['text'].apply(lambda x: x.count(i) )

In [286]:
train_limpio

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,blown,bridge,crush,derailment,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died,yeas
0,"[reason, earthquake, may, forgive, us]",1,0,1,70,1,6,0,0,1,...,0,0,0,0,2,0,0,0,0,0
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,1,40,0,6,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,"[residents, asked, shelter, place, notified, evacuation, shelter, place, orders, expected]",1,0,1,130,1,7,1,3,1,...,0,0,0,0,2,0,0,0,0,0
3,"[people, receive, wildfires, evacuation, orders, california]",1,0,1,60,0,4,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,"[got, sent, photo, ruby, alaska, smoke, wildfires, pours]",1,0,1,90,0,4,2,1,0,...,0,0,0,0,2,0,0,0,0,0
5,"[rockyfire, update, california, hwy, closed, directions, due, lake, county, fire, cafire, wildfires]",1,0,1,110,0,4,1,1,0,...,0,0,0,0,1,0,0,0,0,0
6,"[disaster, heavy, rain, causes, flash, manitou, colorado, springs, areas]",1,0,1,100,0,9,1,4,1,...,0,0,0,0,2,0,0,0,0,0
7,"[im, top, fire]",1,0,1,60,2,3,1,1,1,...,0,0,0,0,2,0,0,0,0,0
8,"[theres, emergency, evacuation, building]",1,0,1,80,2,5,0,3,0,...,0,0,0,0,2,0,0,0,0,0
9,"[im, afraid, tornado, coming, area]",1,0,1,50,1,6,2,1,0,...,0,0,0,0,1,0,0,0,0,0


In [287]:
# Caso mucho mas general: columnas de las palabras mas comunes

## Guardo el train limpio para mas adelante

In [298]:
train_limpio.to_csv('train_limpio.csv', index=False)

## Redes neuronales con tensorFlow (lo pongo aca xq no consegui que ande)

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train) , (x_test, y_test) = mnist.load_data()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_train[0])
plt.show()

In [ ]:
#normalizo la data para reder neuronales para que prediga mejor
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [ ]:
# armo el modelo

## Vectorizo los textos (quiza la parte mas importante de todo el cleaning y apenas lo toque)

In [125]:
train_solo_texto = train_limpio[['id','text']].copy()

In [126]:
train_solo_texto['text'] = train_solo_texto['text'].apply(lambda x: ' '.join(x))

In [145]:
train_solo_texto.head()

,id,text
0,1,reason earthquake may forgive us
1,4,forest fire near la ronge sask canada
2,5,residents asked shelter place notified evacuation shelter place orders expected
3,6,people receive wildfires evacuation orders california
4,7,got sent photo ruby alaska smoke wildfires pours


In [146]:
train_solo_texto.groupby('text')['id'].transform('count')

0       1 
1       1 
2       1 
3       1 
4       1 
       .. 
7608    8 
7609    2 
7610    3 
7611    2 
7612    13
Name: id, Length: 7613, dtype: int64

In [134]:
train_solo_texto.groupby('text').id.count().to_frame().reset_index().nlargest(40,'id')

,text,id
6356,watch airport get sandstorm minute,24
0,,23
6601,wreckage conclusively confirmed malaysia pm investigators families,20
1865,families sue families fatal outbreak disea,19
2581,hot hijacking computers send data sound waves black hat prebreak best,17
917,christian muslims temple mount waving israeli flag via pamela,15
3147,latest homes razed northern california wildfire abc news,13
4131,obama declares disaster saipan obama signs disaster declaration northern marians,13
4451,pic old suicide bomber detonated bomb turkey army trench released,11
568,boy charged manslaughter report boy charged manslaughter fatal sh,10


In [162]:
from sklearn.feature_extraction.text import CountVectorizer
#agregarle n-shingles despues
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(train_solo_texto.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = train_solo_texto.index
data_dtm

,ab,aba,abandon,abandoned,abandonedpics,abandoning,abc,abcnews,abcnorio,abcs,...,zombie,zombies,zone,zones,zonesthank,zouma,zrnf,zumiez,zurich,zxathetis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stemmed = [porter.stem(word) for word in tokens]
#train_solo_texto['stemmed'] = train_solo_texto['text'].apply(lambda x: porter.stem(word) for word in x)

ModuleNotFoundError: No module named 'nltk'

In [47]:
train_solo_texto['stemmed'] = train_solo_texto['text'].apply(lambda x: porter.stem(word) for word in x)

NameError: name 'x' is not defined